# Natural Language Processing with Disaster Tweets

### Team: Robyn Tomson and Otto-Cristofer Vanasaun

#### Downloading and importing essential dependencies and Python packages.

In [234]:
!pip install transformers datasets torch scikit-learn
!pip install tf-keras
!pip install tensorflow
!pip install transformers
!pip install --upgrade tensorflow transformers
import os
import re
import string
from time import time
from collections import Counter

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

from sklearn.metrics import accuracy_score, confusion_matrix

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, LSTM, Dense, Dropout, SpatialDropout1D, Bidirectional
)
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

from transformers import (
    TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification,
    BertTokenizer, TFGPT2LMHeadModel, GPT2Tokenizer, TFAutoModelForSequenceClassification
)

   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   --------------------- ------------------ 5.5/10.1 MB 30.5 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 30.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


  You can safely remove it manually.


#### Disabling warnings

In [2]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

#### Reading data from csv files

In [3]:
dataTrain = pd.read_csv("data/train.csv")
dataTest = pd.read_csv("data/test.csv")
dataSampleSubmit = pd.read_csv("data/sample_submission.csv")

#### Data description and understanding

In [4]:
disasters = []
locations = [] 
total_words = Counter()
for row in dataTrain.itertuples(index=True):
    if pd.notna(row.keyword): 
        if row.keyword not in disasters:
            disasters.append(row.keyword)
    if pd.notna(row.location): 
        if row.location not in locations:
            locations.append(row.location)
            
    words = row.text.split() 
    total_words.update(words)
    
unique_word_count = len(total_words)


print(dataTrain.info())
print("________________________________________")
print("Total words:", sum(total_words.values()))
print("Unique words:", unique_word_count)
print("________________________________________")
print("Number of unique disasters:", len(disasters))
print("________________________________________")
print("Number of unique locations:", len(locations))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
None
________________________________________
Total words: 113461
Unique words: 31924
________________________________________
Number of unique disasters: 221
________________________________________
Number of unique locations: 3341


#### Data preparation

In [239]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
max_len = 1400

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

def preprocess_text(text):
     # Remove links, mentions, hashtags from tweet texts
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    
    # Remove special characters and numbers
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'\d+', '', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove stopwords (words that don't add anything meaningful to the text)
    stopword = set(stopwords.words('english'))
    text = " ".join([word for word in text.split() if word not in stopword])
    
    tokens = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    for token in tokens:
        lemma = lemmatizer.lemmatize(token)
        
    text = ' '.join(tokens)
    return text

def oversampling(data):
    #TODO: apply oversampling and/or undersampling to balance data
    majority_size = data['label'].value_counts().max()
    negative_sample = data[data['label'] == 0]
    positive_sample = data[data['label'] == 1].sample(n = majority_size, replace = True, random_state=42)
    train_random_oversampled = pd.concat([positive_sample, negative_sample])
    train_random_oversampled.sort_index(inplace=True)
    return train_random_oversampled
    
def preprocess_data(data):
    data['text'] = data['text'].apply(preprocess_text)
    label_encoder = LabelEncoder()
    data['label'] = label_encoder.fit_transform(data['target'])

    
    return data, label_encoder

def tokenize_and_prepare(data, max_length=512):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    data['tokens'] = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=max_length, truncation=True))
    token_sequences = pad_sequences(data['tokens'], maxlen=max_length, padding='post')
    return token_sequences


def eeltöötlus(trainset):
    trainset, label_encoder = preprocess_data(trainset)
    train_data, val_data = train_test_split(trainset, test_size=0.2, random_state=42)
    train_data = oversampling(train_data)
    train_labels = train_data['label']
    val_labels = val_data['label']
   # print(train_data['text'])
    train_sequences = tokenize_and_prepare(train_data)
    val_sequences = tokenize_and_prepare(val_data)
    
    train_sequences = pad_sequences(
    train_data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=max_len, truncation=True)),
    maxlen=max_len, padding='post'
)
    return train_sequences, val_sequences, train_labels, val_labels, label_encoder

train_sequences, val_sequences, train_labels, val_labels, label_encoder = eeltöötlus(dataTrain)






[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vanasauo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vanasauo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vanasauo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [240]:
print(train_sequences.shape)
print(train_labels.shape)
print(val_sequences.shape)
print(val_labels.shape)

(6936, 1400)
(6936,)
(1523, 512)
(1523,)


#### Model building

In [241]:
vocab_size = 30522
embedding_dim = 256

modelLSTM = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    SpatialDropout1D(0.4),
    BatchNormalization(),
    Bidirectional(LSTM(4, dropout=0.4, recurrent_dropout=0.3)),
    Dropout(0.4),
    Dense(2, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.1))
])

modelLSTM.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Model training

In [242]:
print(train_sequences.shape)
print(train_labels.shape)
print(val_sequences.shape)
print(val_labels.shape)
train_labels.value_counts()

(6936, 1400)
(6936,)
(1523, 512)
(1523,)


label
1    3468
0    3468
Name: count, dtype: int64

In [243]:
algus = time()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
try:
    history = modelLSTM.fit(
        train_sequences,
        train_labels,
        epochs=5,
        batch_size=32,
        validation_data=(val_sequences, val_labels),
        callbacks=[early_stopping],
        verbose=1
    )
except KeyError as e:
    print("KeyError encountered:", e)

aeg = time()-algus
print("Aega treenimiseks läks {} minutit ja {} sekundit.".format(round(aeg/60), round(aeg%60,2)))

Epoch 1/5
217/217 ━━━━━━━━━━━━━━━━━━━━ 453s 2s/step - accuracy: 0.5412 - loss: 0.9597 - val_accuracy: 0.7183 - val_loss: 0.7937
Epoch 2/5
217/217 ━━━━━━━━━━━━━━━━━━━━ 446s 2s/step - accuracy: 0.6412 - loss: 0.7337 - val_accuracy: 0.7708 - val_loss: 0.6241
Epoch 3/5
217/217 ━━━━━━━━━━━━━━━━━━━━ 446s 2s/step - accuracy: 0.7791 - loss: 0.5602 - val_accuracy: 0.7708 - val_loss: 0.5683
Epoch 4/5
217/217 ━━━━━━━━━━━━━━━━━━━━ 455s 2s/step - accuracy: 0.8274 - loss: 0.4777 - val_accuracy: 0.7708 - val_loss: 0.5656
Epoch 5/5
217/217 ━━━━━━━━━━━━━━━━━━━━ 459s 2s/step - accuracy: 0.8969 - loss: 0.3864 - val_accuracy: 0.7656 - val_loss: 0.5784
Aega treenimiseks läks 38 minutit ja 39.88 sekundit.


In [244]:
X_valLSTM = val_sequences

y_valLSTM = modelLSTM.predict(X_valLSTM)

y_valLSTM = [np.argmax(vastus) for vastus in y_valLSTM]

print("Segadusmaatriks:")
conf_mat = confusion_matrix(val_labels, y_valLSTM)
print(conf_mat)
accuracy = (conf_mat[0][0] + conf_mat[1][1])/(conf_mat[0][0] + conf_mat[0][1] + conf_mat[1][0] + conf_mat[1][1])
precision = conf_mat[0][0] / (conf_mat[0][0] + conf_mat[1][0])
recall = conf_mat[0][0] / (conf_mat[0][0] + conf_mat[0][1])
f1_measure = 2 / ((1/precision) + (1/recall))
print("Accuracy:", accuracy) #TP + TN / TP + FN + FP + TN
print("Precision: ", precision) # TP / TP + FP
print("Recall: ", recall) # TP / TP + FN
print("F1-measure:", f1_measure)

48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 101ms/step
Segadusmaatriks:
[[761 113]
 [236 413]]
Accuracy: 0.7708470124753776
Precision:  0.7632898696088265
Recall:  0.8707093821510298
F1-measure: 0.8134687332977019


#### Model testing

In [247]:
dataTest['text'] = dataTest['text'].apply(preproces)
test_sequences = 
test_sequences = tokenize_and_prepare(test_sequences)
test_sequences = pad_sequences(
    dataTest['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=max_len, truncation=True)),
    maxlen=max_len, padding='post'
)

X_testLSTM = test_sequences

y_predsLSTM = modelLSTM.predict(X_testLSTM)

y_predsLSTM = [np.argmax(vastus) for vastus in y_predsLSTM]

TypeError: expected string or bytes-like object

#### Submition, review, reporting and ...

In [246]:
test_results = pd.DataFrame({'id':dataTest['id'], 'target':y_predsLSTM})
test_results.to_csv("submission12.csv", index=False)

#TODO: Saada f-measure 0.8 peale
#TODO: Ühendada veebilehega et me saaks postrisessil demo näidata
#TODO: See for-loop siia et ta pakuks mis tüüpi disaster on